# Pixelate Club Task 3 COMP9517 20T3

# Import Library and Get Image

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.feature import peak_local_max
from skimage.morphology import watershed

from scipy import ndimage as ndi

from plantcv.plantcv import color_palette


# image list
ara_2012_list = []
ara_2012_label = []

ara_2013_list = []
ara_2013_label = []

tobacco_list = []
tobacco_label = []


# get images

for file_name in glob.glob("Plant/Ara2012/*_rgb.png"):
    ara_2012_list.append(file_name)
    
for file_name in glob.glob("Plant/Ara2012/*_label.png"):
    ara_2012_label.append(file_name)

for file_name in glob.glob('Plant/Ara2013-Canon/*_rgb.png'):
    ara_2013_list.append(file_name)
    
for file_name in glob.glob('Plant/Ara2013-Canon/*_label.png'):
    ara_2013_label.append(file_name)

for file_name in glob.glob('Plant/Tobacco/*_rgb.png'):
    tobacco_list.append(file_name)
    
for file_name in glob.glob('Plant/Tobacco/*_label.png'):
    tobacco_label.append(file_name)


# Perform Individual Leaf Segmentation

In [2]:
# get mean value of a in lab for each pixel
def mean_colour_a(img):
    
    # if no pixel, return max allowable
    if len(img.flatten()) == 0:
        return 255
        
    return np.mean(img[img > 0], dtype=np.float64)

def task3(img_rgb, img_threshold, min_distance):
    
    # (1) GENERATE SUPER PIXEL      
    
    # convert to lab colour space 
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
    
    # custom number of segment
    number_of_segments = 1000
    
    # apply slic - Simple Linear Iterative Clustering
    ## max_iter - no. of iteration
    ## sigma - width of gaussian filter
    segments = slic(img_lab, n_segments = number_of_segments, max_iter = 3, sigma = 3)
          
    # (2) SEGMENT PLANT FROM BACKGROUND
    
    # extract 'a' range of plant
    img_lab_a = img_lab[:,:,1]

    # contrast stretching
    img_lab_a = (img_lab_a - np.min(img_lab_a)) * (255 / (np.max(img_lab_a) - np.min(img_lab_a)))
    img_lab_a = img_lab_a.astype(np.uint8)

    # median blur to remove minor noise
    img_lab_a = cv2.medianBlur(img_lab_a, 5)
    
    # black blank image
    img_segmented = np.zeros_like(img_rgb, np.uint8)

    # custom threshold of mean colour of a (a- = green)
    threshold = img_threshold

    # go thru each segment ID
    for segVal in np.unique(segments):

        # get all pixel that has same segmented ID
        mask = np.zeros(img_rgb.shape[:2])
        mask[segments == segVal] = 255

        # Mask of LAB (for calculation)
        img_lab_mask = np.zeros_like(img_rgb, np.uint8)
        img_lab_mask[mask > 0] = img_lab[mask > 0]

        # Mask of RGB (for display)
        img_rgb_mask = np.zeros_like(img_rgb, np.uint8)
        img_rgb_mask[mask > 0] = img_rgb[mask > 0]

        # Get mean value of "a" of pixels with the same ID
        mean = mean_colour_a(img_lab_mask) - 128

        # If below the threshold, it's a plant super pixel
        if mean <= threshold:
            img_segmented += img_rgb_mask
    
    # (3) IMAGE MASK
    img_gray = cv2.cvtColor(img_segmented, cv2.COLOR_RGB2GRAY)
    img_mask = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # (4) FIND LOCAL MAXIMA

    # for each labelled pixel, find distance to the nearest background pixel
    dist_transform = cv2.distanceTransformWithLabels(img_mask, cv2.DIST_L2, maskSize=0)[0]

    # find local maxima
    localMax = peak_local_max(dist_transform, indices=False, min_distance = min_distance, labels=img_mask)

    # (5) WATERSHED SEGMENTATION

    # label
    markers = ndi.label(localMax, structure=np.ones((3, 3)))[0]
    dist_transform1 = -dist_transform
    labels = watershed(dist_transform1, markers, mask=img_mask)

    # colour image label
    img_final =np.zeros_like(img_rgb, np.uint8)
    for x in np.unique(labels[labels > 0]):

        rand_color = color_palette(len(np.unique(labels)))
        img_final[labels == x] = rand_color[x]
        
    return img_final

# Metrics

In [3]:
def convert_bool(g):
    m = g.copy()
    m[g >0] = True
    m[g == 0] = False
    return m

# Similarity dice measures
# calculate dice
def dice_calc (ground_th, ours):

    numerator = 2 * np.sum(ground_th * ours)
    denominator = np.sum(ground_th) + np.sum(ours)

    dice = float(numerator) / float(denominator)
    
    return dice


# calculate best dice -------
def best_dice (groud_th, ours):

    ground_th_idx = list(set(np.unique(groud_th)).difference([0]))
    ours_idx = list(set(np.unique(ours)).difference([0]))

    best_dice_collect = []
    for i_gt in ground_th_idx:
        
        gt = (groud_th == i_gt).astype('bool')
        
        dice_collect = []
        
        for i_ours in ours_idx:
            
            ors = (ours == i_ours).astype('bool')

            dice = dice_calc(gt, ors)
            
            dice_collect.append(dice)
        

        best_dice = np.max(dice_collect)
        best_dice_collect.append(best_dice)

    best_d = np.mean(best_dice_collect)

    return best_d


# symmetric best dice calculation ----------
def symm_best_dice (ground_th, ours):

    d1 = best_dice(ground_th, ours)
    d2 = best_dice(ours, ground_th)
    
    arr = np.array ([d1, d2])
    symm_best = np.min(arr)
    return symm_best

# Ara 2012 Performance

In [4]:

dice_calc_result = []
symm_best_dice_result = []

for x in range(len(ara_2012_list)):
    
    print(ara_2012_list[x])
    
    img = cv2.imread(ara_2012_list[x])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    label = cv2.imread(ara_2012_list[x])
    img_label_rgb = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
    
    img_result = task3(img_rgb, -5, 5)
    
    img_result = convert_bool(img_result)
    img_label = convert_bool(img_label_rgb)
    
    dice_calc_score = dice_calc(img_label, img_result)
    best_dice_score = symm_best_dice(img_label, img_result)
    
    print("Dice Score:", dice_calc(img_label, img_result))
    print("Best Dice Symmetry Score: ", symm_best_dice(img_label, img_result))
    print()
    
    dice_calc_result.append(dice_calc_score)
    symm_best_dice_result.append(dice_calc_score)

Plant/Ara2012\ara2012_plant001_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.6059727457233981
Best Dice Symmetry Score:  0.6059727457233981

Plant/Ara2012\ara2012_plant002_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.42645698427382056
Best Dice Symmetry Score:  0.42645698427382056

Plant/Ara2012\ara2012_plant003_rgb.png


...

c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


Dice Score: 0.29913587998855246
Best Dice Symmetry Score:  0.29913587998855246

Plant/Ara2012\ara2012_plant120_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.4270597578546563
Best Dice Symmetry Score:  0.4270597578546563



c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


In [5]:
dice_calc_result = np.array(dice_calc_result)
symm_best_dice_result = np.array(symm_best_dice_result)

print("Mean Dice Calc score for Ara 2012:",  np.mean(dice_calc_result))
print("Mean Best Dice Symmetry score for Ara 2012:", np.mean(symm_best_dice_result))

Mean Dice Calc score for Ara 2012: 0.4127831021726468
Mean Best Dice Symmetry score for Ara 2012: 0.4127831021726468


# Ara 2013-Canon

In [6]:

dice_calc_result = []
symm_best_dice_result = []

for x in range(len(ara_2013_list)):
    
    print(ara_2013_list[x])
    
    img = cv2.imread(ara_2013_list[x])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    label = cv2.imread(ara_2013_list[x])
    img_label_rgb = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
    
    img_result = task3(img_rgb, -20, 5)
    
    img_result = convert_bool(img_result)
    img_label = convert_bool(img_label_rgb)
    
    dice_calc_score = dice_calc(img_label, img_result)
    best_dice_score = symm_best_dice(img_label, img_result)
    
    print("Dice Score:", dice_calc(img_label, img_result))
    print("Best Dice Symmetry Score: ", symm_best_dice(img_label, img_result))
    
    dice_calc_result.append(dice_calc_score)
    symm_best_dice_result.append(dice_calc_score)

Plant/Ara2013-Canon\ara2013_plant001_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.6158329569132582
Best Dice Symmetry Score:  0.6158329569132582
Plant/Ara2013-Canon\ara2013_plant002_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` 

Dice Score: 0.5849507878295911
Best Dice Symmetry Score:  0.5849507878295911
Plant/Ara2013-Canon\ara2013_plant003_rgb.png
Dice Score: 0.6526489762188963
Best Dice Symmetry Score:  0.6526489762188963
Plant/Ara2013-Canon\ara2013_plant004_rgb.png


...

c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.5375026512741266
Best Dice Symmetry Score:  0.5375026512741266
Plant/Ara2013-Canon\ara2013_plant164_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


Dice Score: 0.5062881636944478
Best Dice Symmetry Score:  0.5062881636944478
Plant/Ara2013-Canon\ara2013_plant165_rgb.png
Dice Score: 0.5797495917256397
Best Dice Symmetry Score:  0.5797495917256397


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


In [7]:
dice_calc_result = np.array(dice_calc_result)
symm_best_dice_result = np.array(symm_best_dice_result)

print("Mean Dice Calc score for Ara 2013:",  np.mean(dice_calc_result))
print("Mean Best Dice Symmetry score for Ara 2013:", np.mean(symm_best_dice_result))

Mean Dice Calc score for Ara 2013: 0.5608326470986074
Mean Best Dice Symmetry score for Ara 2013: 0.5608326470986074


# Tobacco

In [12]:

dice_calc_result = []
symm_best_dice_result = []

for x in range(10):
    
    print(tobacco_list[x])
    
    img = cv2.imread(tobacco_list[x])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    label = cv2.imread(tobacco_list[x])
    img_label_rgb = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
    
    scale_percent = 30 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    # resize image
    img_rgb = cv2.resize(img_rgb, dim, interpolation = cv2.INTER_AREA)
    img_label_rgb = cv2.resize(img_label_rgb, dim, interpolation = cv2.INTER_AREA)
    
    img_result = task3(img_rgb, -20, 5)
    
    img_result = convert_bool(img_result)
    img_label = convert_bool(img_label_rgb)
    
    dice_calc_score = dice_calc(img_label, img_result)
    best_dice_score = symm_best_dice(img_label, img_result)
    
    print("Dice Score:", dice_calc(img_label, img_result))
    print("Best Dice Symmetry Score: ", symm_best_dice(img_label, img_result))
    
    dice_calc_result.append(dice_calc_score)
    symm_best_dice_result.append(dice_calc_score)

Plant/Tobacco\tobacco_plant001_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


Dice Score: 0.32793320236485296
Best Dice Symmetry Score:  0.32793320236485296
Plant/Tobacco\tobacco_plant002_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


Dice Score: 0.42020629277683164
Best Dice Symmetry Score:  0.42020629277683164
Plant/Tobacco\tobacco_plant003_rgb.png


...

Dice Score: 0.29343273559016514
Best Dice Symmetry Score:  0.29343273559016514
Plant/Tobacco\tobacco_plant009_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


Dice Score: 0.30783075716698677
Best Dice Symmetry Score:  0.30783075716698677
Plant/Tobacco\tobacco_plant010_rgb.png


c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
c:\users\user\pycharmprojects\comp9517\venv\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.


Dice Score: 0.3521688708123416
Best Dice Symmetry Score:  0.3521688708123416


In [11]:
dice_calc_result = np.array(dice_calc_result)
symm_best_dice_result = np.array(symm_best_dice_result)

print("Mean Dice Calc score for Tobacco:",  np.mean(dice_calc_result))
print("Mean Best Dice Symmetry score for Tobacco:", np.mean(symm_best_dice_result))

Mean Dice Calc score for Tobacco: 0.2510662490396306
Mean Best Dice Symmetry score for Tobacco: 0.2510662490396306
